In [48]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [49]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [50]:
X_train = x_train / 255
X_test = x_test / 255

In [51]:
X_train = tf.reshape(tf.cast(X_train, tf.float32), [-1, 28*28])
X_test = tf.reshape(tf.cast(X_test, tf.float32), [-1, 28*28])

In [52]:
y_train = to_categorical(y_train, 10)  # OneHot

In [53]:
class DenseNN(tf.Module):
    def __init__(self, outputs, activate="relu"):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False

    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name="b")

            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.fl_init = True

        y = x @ self.w + self.b

        if self.activate == "relu":
            return tf.nn.relu(y)
        elif self.activate == "softmax":
            return tf.nn.softmax(y)

        return y

In [54]:
class SequentialModule(tf.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = DenseNN(128)
        self.layer_2 = DenseNN(10, activate='softmax')

    def __call__(self, x):
        return self.layer_2(self.layer_1(x))

In [55]:
model = SequentialModule()

In [56]:
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))

In [57]:
opt = tf.optimizers.Adam(learning_rate=0.001)

In [58]:
BATCH_SIZE = 32
EPOCHS = 10
TOTAL = X_train.shape[0]

In [59]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [60]:
for n in range(EPOCHS):
    loss = 0

    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model(x_batch))

        loss += f_loss
        grads = tape.gradient(f_loss, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))

    print(loss.numpy())

504.75067
217.70613
147.45967
107.41559
81.319
63.674854
51.554867
39.879166
32.249752
25.778196


In [63]:
y = model(X_test)
y2 = tf.argmax(y, axis=1).numpy()
# 1 var. handle accuracy
acc = len(y_test[y_test==y2])/y_test.shape[0] * 100
acc

# 2 var. tf accuracy
# acc = tf.metrics.Accuracy()
# acc.update_state(y_test, y2)
# print(acc.result().numpy() * 100)

97.77

In [68]:
for i in model.submodules:
    print(i)